# Module 07: Trend and Convergence Mathematics

**Difficulty**: ⭐⭐ Intermediate  
**Estimated Time**: 60 minutes  
**Prerequisites**: 
- Module 04: Moving Averages (EMA calculation)
- Module 05: Momentum Mathematics
- Basic understanding of price trends

## Learning Objectives

By the end of this notebook, you will be able to:
1. **Calculate MACD** from scratch using the difference between two EMAs
2. **Understand the Signal Line** as a 9-period EMA of the MACD
3. **Interpret the Histogram** as a measure of convergence/divergence
4. **Explain why MACD captures both momentum AND trend** mathematically
5. **Identify MACD crossovers** and their trading implications
6. **Understand MACD divergence** as an early warning signal

---

## What is MACD?

**MACD** (Moving Average Convergence Divergence) is one of the most popular technical indicators. Despite its complex name, the mathematics is straightforward:

$$\text{MACD} = \text{EMA}_{12} - \text{EMA}_{26}$$

That's it! The MACD is simply the **difference** between a fast EMA (12-period) and a slow EMA (26-period).

### Why Does This Work?

The MACD captures **two things simultaneously**:

1. **Trend**: The sign of MACD tells you the trend direction
   - If MACD > 0: Fast EMA > Slow EMA → **Uptrend**
   - If MACD < 0: Fast EMA < Slow EMA → **Downtrend**

2. **Momentum**: The magnitude of MACD tells you trend strength
   - Large positive MACD → Strong uptrend
   - Large negative MACD → Strong downtrend
   - MACD near 0 → Weak trend or consolidation

### The Three Components

The complete MACD indicator has **three components**:

1. **MACD Line**: $\text{EMA}_{12} - \text{EMA}_{26}$
2. **Signal Line**: $\text{EMA}_9(\text{MACD})$ ← This is a 9-period EMA of the MACD line
3. **Histogram**: $\text{MACD} - \text{Signal}$ ← Shows the difference visually

Let's implement all three from scratch using **Malaysian stocks**!

---

In [ ]:
# Setup and imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings

# Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')
np.random.seed(42)

# Plot settings for better visibility
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully")
print(f"Today's date: {datetime.now().strftime('%Y-%m-%d')}")

## 1. MACD Calculation: The Difference Between EMAs

Let's start by downloading Malaysian stock data and calculating the MACD line.

### Mathematical Steps:

1. Calculate 12-period EMA of closing prices
2. Calculate 26-period EMA of closing prices
3. MACD = EMA(12) - EMA(26)

**Recall**: The EMA formula from Module 04:

$$\text{EMA}_{\text{today}} = \alpha \times \text{Price}_{\text{today}} + (1 - \alpha) \times \text{EMA}_{\text{yesterday}}$$

where $\alpha = \frac{2}{n + 1}$ and $n$ is the period.

---

In [ ]:
# Download Maybank data (a stable Malaysian bank stock)
ticker = '1155.KL'  # Maybank
start_date = '2023-01-01'
end_date = '2024-01-01'

print(f"Downloading {ticker} data from {start_date} to {end_date}...")
maybank = yf.download(ticker, start=start_date, end=end_date, progress=False)

# Validate data
print(f"\n✓ Downloaded {len(maybank)} trading days")
print(f"Price range: RM {maybank['Close'].min():.2f} - RM {maybank['Close'].max():.2f}")

# Display first few rows
maybank.head()

In [ ]:
# Calculate MACD from scratch
def calculate_ema(prices, period):
    """
    Calculate Exponential Moving Average.
    
    Parameters:
    -----------
    prices : pd.Series
        Price series
    period : int
        EMA period (e.g., 12, 26)
    
    Returns:
    --------
    pd.Series : EMA values
    """
    # Smoothing factor: α = 2 / (period + 1)
    alpha = 2 / (period + 1)
    
    # Use pandas ewm for efficient calculation
    ema = prices.ewm(alpha=alpha, adjust=False).mean()
    
    return ema


def calculate_macd(prices, fast=12, slow=26, signal=9):
    """
    Calculate MACD, Signal Line, and Histogram.
    
    Parameters:
    -----------
    prices : pd.Series
        Closing prices
    fast : int
        Fast EMA period (default: 12)
    slow : int
        Slow EMA period (default: 26)
    signal : int
        Signal line EMA period (default: 9)
    
    Returns:
    --------
    pd.DataFrame : DataFrame with MACD, Signal, and Histogram columns
    """
    # Step 1: Calculate fast and slow EMAs
    ema_fast = calculate_ema(prices, fast)
    ema_slow = calculate_ema(prices, slow)
    
    # Step 2: MACD line = Fast EMA - Slow EMA
    macd_line = ema_fast - ema_slow
    
    # Step 3: Signal line = 9-period EMA of MACD line
    signal_line = calculate_ema(macd_line, signal)
    
    # Step 4: Histogram = MACD - Signal
    histogram = macd_line - signal_line
    
    # Combine into DataFrame
    result = pd.DataFrame({
        'MACD': macd_line,
        'Signal': signal_line,
        'Histogram': histogram,
        'EMA_12': ema_fast,  # For reference
        'EMA_26': ema_slow   # For reference
    })
    
    return result


# Calculate MACD for Maybank
macd_data = calculate_macd(maybank['Close'])

print("MACD Components Calculated:")
print("="*50)
print(macd_data.tail(10))
print("\nInterpretation of last value:")
print(f"MACD: {macd_data['MACD'].iloc[-1]:.4f}")
print(f"Signal: {macd_data['Signal'].iloc[-1]:.4f}")
print(f"Histogram: {macd_data['Histogram'].iloc[-1]:.4f}")

### Understanding the Output

- **MACD**: The difference between 12-EMA and 26-EMA
  - Positive MACD → Fast EMA above slow EMA → Bullish
  - Negative MACD → Fast EMA below slow EMA → Bearish

- **Signal**: The 9-period EMA of the MACD line
  - Acts as a "trigger" for buy/sell signals
  - Smoother than the MACD line

- **Histogram**: MACD - Signal
  - Shows the **distance** between MACD and Signal
  - When histogram crosses zero → MACD crosses Signal → Trading signal!

---

In [ ]:
# Visualize MACD components
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Panel 1: Price with EMAs
axes[0].plot(maybank.index, maybank['Close'], label='Close Price', color='black', linewidth=1.5)
axes[0].plot(macd_data.index, macd_data['EMA_12'], label='EMA(12)', color='blue', linewidth=1.2)
axes[0].plot(macd_data.index, macd_data['EMA_26'], label='EMA(26)', color='red', linewidth=1.2)
axes[0].set_ylabel('Price (RM)', fontsize=11)
axes[0].set_title('Maybank (1155.KL) - Price and EMAs', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Panel 2: MACD and Signal Line
axes[1].plot(macd_data.index, macd_data['MACD'], label='MACD Line', color='blue', linewidth=1.5)
axes[1].plot(macd_data.index, macd_data['Signal'], label='Signal Line', color='red', linewidth=1.5)
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
axes[1].fill_between(macd_data.index, macd_data['MACD'], macd_data['Signal'], 
                      where=(macd_data['MACD'] >= macd_data['Signal']), 
                      color='green', alpha=0.3, label='MACD > Signal (Bullish)')
axes[1].fill_between(macd_data.index, macd_data['MACD'], macd_data['Signal'], 
                      where=(macd_data['MACD'] < macd_data['Signal']), 
                      color='red', alpha=0.3, label='MACD < Signal (Bearish)')
axes[1].set_ylabel('MACD Value', fontsize=11)
axes[1].set_title('MACD and Signal Line', fontsize=12, fontweight='bold')
axes[1].legend(loc='best')
axes[1].grid(True, alpha=0.3)

# Panel 3: Histogram
colors = ['green' if val >= 0 else 'red' for val in macd_data['Histogram']]
axes[2].bar(macd_data.index, macd_data['Histogram'], color=colors, alpha=0.6, width=1)
axes[2].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[2].set_ylabel('Histogram', fontsize=11)
axes[2].set_xlabel('Date', fontsize=11)
axes[2].set_title('MACD Histogram (MACD - Signal)', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Chart Interpretation:")
print("="*50)
print("Top Panel: Price with 12-EMA (blue) and 26-EMA (red)")
print("  - When blue > red: MACD is positive")
print("  - When blue < red: MACD is negative")
print("\nMiddle Panel: MACD and Signal Lines")
print("  - Green shading: MACD > Signal (bullish momentum)")
print("  - Red shading: MACD < Signal (bearish momentum)")
print("\nBottom Panel: Histogram")
print("  - Green bars: MACD > Signal")
print("  - Red bars: MACD < Signal")
print("  - Histogram crossing zero = Trading signal!")

## 2. Signal Line: The Trigger

The **Signal Line** is simply a **9-period EMA of the MACD line**. But why is this important?

### Mathematical Reasoning:

The Signal Line serves as a **smoothed version** of the MACD. Think of it as a "slow follower" of the MACD:

$$\text{Signal} = \text{EMA}_9(\text{MACD})$$

### Why 9 Periods?

The choice of 9 periods is somewhat arbitrary (like the 14-period RSI), but it has become the **standard** because:

1. **Fast enough** to generate timely signals
2. **Smooth enough** to filter out noise
3. **Historical validation** - traders have used it for decades

### Trading Signals from Crossovers:

| Condition | Signal | Interpretation |
|-----------|--------|----------------|
| MACD crosses **above** Signal | **Buy** | Momentum turning bullish |
| MACD crosses **below** Signal | **Sell** | Momentum turning bearish |
| MACD >> Signal (large gap) | **Strong uptrend** | Consider taking profits |
| MACD << Signal (large gap) | **Strong downtrend** | Consider buying dip |

Let's identify these crossovers in real data!

---

In [ ]:
# Detect MACD crossovers
def detect_macd_crossovers(macd_data):
    """
    Detect bullish and bearish MACD crossovers.
    
    Parameters:
    -----------
    macd_data : pd.DataFrame
        DataFrame with 'MACD' and 'Signal' columns
    
    Returns:
    --------
    pd.DataFrame : Original data with 'Crossover' column added
    """
    df = macd_data.copy()
    
    # Create a column to track the relationship
    # 1 if MACD > Signal, -1 if MACD < Signal
    df['Position'] = np.where(df['MACD'] > df['Signal'], 1, -1)
    
    # Detect changes in position (crossovers)
    df['Crossover'] = df['Position'].diff()
    
    # Crossover = 2 means bullish crossover (from -1 to 1)
    # Crossover = -2 means bearish crossover (from 1 to -1)
    df['Signal_Type'] = ''
    df.loc[df['Crossover'] == 2, 'Signal_Type'] = 'Bullish'
    df.loc[df['Crossover'] == -2, 'Signal_Type'] = 'Bearish'
    
    return df


# Detect crossovers
macd_with_signals = detect_macd_crossovers(macd_data)

# Get all crossover points
bullish_crossovers = macd_with_signals[macd_with_signals['Signal_Type'] == 'Bullish']
bearish_crossovers = macd_with_signals[macd_with_signals['Signal_Type'] == 'Bearish']

print("MACD Crossover Signals Detected:")
print("="*70)
print(f"\n🟢 Bullish Crossovers (Buy Signals): {len(bullish_crossovers)}")
print(bullish_crossovers[['MACD', 'Signal', 'Histogram']].to_string())

print(f"\n🔴 Bearish Crossovers (Sell Signals): {len(bearish_crossovers)}")
print(bearish_crossovers[['MACD', 'Signal', 'Histogram']].to_string())

print(f"\n📈 Total Trading Signals: {len(bullish_crossovers) + len(bearish_crossovers)}")

In [ ]:
# Visualize crossovers on price chart
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Panel 1: Price with buy/sell signals
axes[0].plot(maybank.index, maybank['Close'], label='Close Price', color='black', linewidth=1.5)

# Mark bullish crossovers (buy signals)
for date in bullish_crossovers.index:
    price = maybank.loc[date, 'Close']
    axes[0].scatter(date, price, color='green', marker='^', s=200, zorder=5, 
                    edgecolors='black', linewidths=1.5)

# Mark bearish crossovers (sell signals)
for date in bearish_crossovers.index:
    price = maybank.loc[date, 'Close']
    axes[0].scatter(date, price, color='red', marker='v', s=200, zorder=5,
                    edgecolors='black', linewidths=1.5)

axes[0].set_ylabel('Price (RM)', fontsize=11)
axes[0].set_title('Maybank - MACD Trading Signals', fontsize=12, fontweight='bold')
axes[0].legend(['Price', 'Buy Signal (Bullish Crossover)', 'Sell Signal (Bearish Crossover)'], 
               loc='best', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Panel 2: MACD with crossover markers
axes[1].plot(macd_with_signals.index, macd_with_signals['MACD'], 
             label='MACD', color='blue', linewidth=1.5)
axes[1].plot(macd_with_signals.index, macd_with_signals['Signal'], 
             label='Signal', color='red', linewidth=1.5)
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)

# Mark crossover points
axes[1].scatter(bullish_crossovers.index, bullish_crossovers['MACD'], 
                color='green', marker='^', s=150, zorder=5, label='Bullish Crossover')
axes[1].scatter(bearish_crossovers.index, bearish_crossovers['MACD'], 
                color='red', marker='v', s=150, zorder=5, label='Bearish Crossover')

axes[1].set_ylabel('MACD Value', fontsize=11)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_title('MACD and Signal Line with Crossovers', fontsize=12, fontweight='bold')
axes[1].legend(loc='best', fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Signal Interpretation:")
print("="*70)
print("🟢 Green triangles pointing UP: Buy signals (MACD crossed above Signal)")
print("🔴 Red triangles pointing DOWN: Sell signals (MACD crossed below Signal)")
print("\n⚠️  Important: Not all crossovers are profitable!")
print("    - Check trend context (is there a clear trend?)")
print("    - Combine with other indicators for confirmation")
print("    - MACD works best in trending markets, not sideways")

## 3. Histogram: Visual Convergence and Divergence

The **Histogram** is the most visually intuitive component:

$$\text{Histogram} = \text{MACD} - \text{Signal}$$

### Why is it Useful?

The histogram shows the **distance** between MACD and Signal. This reveals:

1. **Convergence**: Histogram getting smaller → MACD and Signal approaching each other → Potential crossover soon
2. **Divergence**: Histogram getting larger → MACD and Signal moving apart → Trend strengthening

### Histogram States:

| Histogram | MACD vs Signal | Interpretation |
|-----------|----------------|----------------|
| **Positive and growing** | MACD > Signal, gap widening | Bullish momentum accelerating |
| **Positive but shrinking** | MACD > Signal, gap narrowing | Bullish momentum weakening |
| **Crosses from - to +** | MACD crosses above Signal | **BUY SIGNAL** |
| **Negative and growing** | MACD < Signal, gap widening | Bearish momentum accelerating |
| **Negative but shrinking** | MACD < Signal, gap narrowing | Bearish momentum weakening |
| **Crosses from + to -** | MACD crosses below Signal | **SELL SIGNAL** |

---

In [ ]:
# Analyze histogram momentum
def analyze_histogram_momentum(macd_data):
    """
    Analyze whether histogram is growing or shrinking.
    
    Parameters:
    -----------
    macd_data : pd.DataFrame
        DataFrame with 'Histogram' column
    
    Returns:
    --------
    pd.DataFrame : Data with momentum analysis columns
    """
    df = macd_data.copy()
    
    # Calculate change in histogram
    df['Hist_Change'] = df['Histogram'].diff()
    
    # Determine momentum state
    df['Momentum_State'] = ''
    
    # Positive histogram
    df.loc[(df['Histogram'] > 0) & (df['Hist_Change'] > 0), 'Momentum_State'] = 'Bullish Accelerating'
    df.loc[(df['Histogram'] > 0) & (df['Hist_Change'] < 0), 'Momentum_State'] = 'Bullish Weakening'
    
    # Negative histogram
    df.loc[(df['Histogram'] < 0) & (df['Hist_Change'] < 0), 'Momentum_State'] = 'Bearish Accelerating'
    df.loc[(df['Histogram'] < 0) & (df['Hist_Change'] > 0), 'Momentum_State'] = 'Bearish Weakening'
    
    return df


# Analyze momentum
macd_momentum = analyze_histogram_momentum(macd_data)

# Show recent momentum states
print("Recent Histogram Momentum Analysis:")
print("="*80)
print(macd_momentum[['MACD', 'Signal', 'Histogram', 'Hist_Change', 'Momentum_State']].tail(15).to_string())

# Count momentum states
print("\n📊 Momentum State Distribution:")
print(macd_momentum['Momentum_State'].value_counts())

In [ ]:
# Visualize histogram with momentum states
fig, ax = plt.subplots(figsize=(14, 6))

# Color bars based on momentum state
colors = []
for idx, row in macd_momentum.iterrows():
    if row['Momentum_State'] == 'Bullish Accelerating':
        colors.append('darkgreen')
    elif row['Momentum_State'] == 'Bullish Weakening':
        colors.append('lightgreen')
    elif row['Momentum_State'] == 'Bearish Accelerating':
        colors.append('darkred')
    elif row['Momentum_State'] == 'Bearish Weakening':
        colors.append('lightcoral')
    else:
        colors.append('gray')

ax.bar(macd_momentum.index, macd_momentum['Histogram'], color=colors, alpha=0.7, width=1)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1.5)
ax.set_ylabel('Histogram Value', fontsize=11)
ax.set_xlabel('Date', fontsize=11)
ax.set_title('MACD Histogram with Momentum States', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='darkgreen', label='Bullish Accelerating'),
    Patch(facecolor='lightgreen', label='Bullish Weakening'),
    Patch(facecolor='lightcoral', label='Bearish Weakening'),
    Patch(facecolor='darkred', label='Bearish Accelerating')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=9)

plt.tight_layout()
plt.show()

print("\n📊 Histogram Interpretation:")
print("="*70)
print("Dark Green bars: Bullish momentum getting STRONGER")
print("Light Green bars: Bullish momentum getting WEAKER (watch for reversal)")
print("Light Red bars: Bearish momentum getting WEAKER (potential bottom)")
print("Dark Red bars: Bearish momentum getting STRONGER")
print("\n💡 Trading Insight:")
print("   Look for transitions from dark to light colors - these often precede reversals!")

## 4. Why MACD Shows Both Momentum AND Trend

This is the **mathematical beauty** of MACD. Let's break it down:

### Momentum Component:

The MACD measures the **rate of change** between two EMAs:

$$\text{MACD} = \text{EMA}_{12} - \text{EMA}_{26}$$

- When prices are **rising fast**: EMA(12) increases faster than EMA(26) → MACD increases → **Positive momentum**
- When prices are **falling fast**: EMA(12) decreases faster than EMA(26) → MACD decreases → **Negative momentum**

### Trend Component:

The **sign** of MACD tells you the trend:

- **MACD > 0** means EMA(12) > EMA(26) → Short-term average above long-term average → **Uptrend**
- **MACD < 0** means EMA(12) < EMA(26) → Short-term average below long-term average → **Downtrend**

### The Dual Nature:

| MACD Value | Trend | Histogram | Momentum | Overall Signal |
|------------|-------|-----------|----------|----------------|
| Large positive | Uptrend | Positive & growing | Accelerating up | **Strong buy** |
| Small positive | Weak uptrend | Positive but shrinking | Decelerating | **Watch for reversal** |
| Small negative | Weak downtrend | Negative but shrinking | Losing bearish momentum | **Potential buy** |
| Large negative | Downtrend | Negative & growing | Accelerating down | **Strong sell** |

Let's compare MACD with pure momentum (ROC) and pure trend (SMA) to see the difference!

---

In [ ]:
# Compare MACD with ROC (momentum) and SMA crossover (trend)
def calculate_roc(prices, period=12):
    """Calculate Rate of Change (pure momentum indicator)"""
    roc = ((prices - prices.shift(period)) / prices.shift(period)) * 100
    return roc

def calculate_sma_signal(prices, fast=12, slow=26):
    """Calculate SMA crossover signal (pure trend indicator)"""
    sma_fast = prices.rolling(window=fast).mean()
    sma_slow = prices.rolling(window=slow).mean()
    sma_diff = sma_fast - sma_slow
    return sma_diff

# Calculate comparison indicators
comparison = pd.DataFrame({
    'Price': maybank['Close'],
    'MACD': macd_data['MACD'],
    'ROC': calculate_roc(maybank['Close'], period=12),
    'SMA_Diff': calculate_sma_signal(maybank['Close'], fast=12, slow=26)
})

# Normalize for comparison (scale to -1 to 1)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
comparison['MACD_norm'] = scaler.fit_transform(comparison[['MACD']])
comparison['ROC_norm'] = scaler.fit_transform(comparison[['ROC']].fillna(0))
comparison['SMA_norm'] = scaler.fit_transform(comparison[['SMA_Diff']].fillna(0))

# Visualize comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Panel 1: Price
axes[0].plot(comparison.index, comparison['Price'], color='black', linewidth=1.5)
axes[0].set_ylabel('Price (RM)', fontsize=11)
axes[0].set_title('Maybank Price', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Panel 2: Normalized indicators
axes[1].plot(comparison.index, comparison['MACD_norm'], 
             label='MACD (Trend + Momentum)', color='blue', linewidth=2)
axes[1].plot(comparison.index, comparison['ROC_norm'], 
             label='ROC (Pure Momentum)', color='orange', linewidth=1.5, linestyle='--')
axes[1].plot(comparison.index, comparison['SMA_norm'], 
             label='SMA Diff (Pure Trend)', color='green', linewidth=1.5, linestyle=':')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].set_ylabel('Normalized Value', fontsize=11)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_title('MACD vs Pure Momentum vs Pure Trend', fontsize=12, fontweight='bold')
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Indicator Comparison:")
print("="*70)
print("MACD (Blue solid): Captures BOTH trend direction AND momentum strength")
print("ROC (Orange dashed): Captures only momentum (rate of change)")
print("SMA Diff (Green dotted): Captures only trend (moving average difference)")
print("\n💡 Key Insight:")
print("   MACD combines the best of both worlds - it's smoother than ROC")
print("   but more responsive than SMA crossover!")

## 5. MACD Divergence: Early Warning System

**Divergence** is one of the most powerful MACD signals. It occurs when price and MACD move in opposite directions.

### Types of Divergence:

**1. Bullish Divergence** (Reversal signal - potential bottom)
- Price makes **lower lows** (downtrend continuing)
- MACD makes **higher lows** (momentum weakening)
- **Interpretation**: Selling pressure is decreasing → Potential reversal up

**2. Bearish Divergence** (Reversal signal - potential top)
- Price makes **higher highs** (uptrend continuing)
- MACD makes **lower highs** (momentum weakening)
- **Interpretation**: Buying pressure is decreasing → Potential reversal down

### Why Does Divergence Work?

Divergence reveals **momentum exhaustion**:
- Price can keep moving in a direction due to inertia
- But the **rate of change** (captured by MACD) is slowing
- Eventually, the trend reverses when momentum is gone

Let's detect divergence programmatically!

---

In [ ]:
# Detect MACD divergence
def find_peaks_and_troughs(series, order=5):
    """
    Find local peaks and troughs in a series.
    
    Parameters:
    -----------
    series : pd.Series
        Data series
    order : int
        How many points on each side to consider for peak/trough
    
    Returns:
    --------
    tuple : (peaks, troughs) as boolean series
    """
    from scipy.signal import argrelextrema
    
    # Find local maxima (peaks)
    peaks_idx = argrelextrema(series.values, np.greater, order=order)[0]
    peaks = pd.Series(False, index=series.index)
    peaks.iloc[peaks_idx] = True
    
    # Find local minima (troughs)
    troughs_idx = argrelextrema(series.values, np.less, order=order)[0]
    troughs = pd.Series(False, index=series.index)
    troughs.iloc[troughs_idx] = True
    
    return peaks, troughs


def detect_divergence(price, macd, order=5):
    """
    Detect bullish and bearish divergence.
    
    Returns:
    --------
    dict : Contains 'bullish' and 'bearish' divergence points
    """
    # Find peaks and troughs
    price_peaks, price_troughs = find_peaks_and_troughs(price, order=order)
    macd_peaks, macd_troughs = find_peaks_and_troughs(macd, order=order)
    
    bullish_div = []
    bearish_div = []
    
    # Detect bullish divergence (price lower lows, MACD higher lows)
    trough_dates = price[price_troughs].index
    for i in range(1, len(trough_dates)):
        date1, date2 = trough_dates[i-1], trough_dates[i]
        
        # Check if MACD troughs exist at similar dates
        if date1 in macd.index and date2 in macd.index:
            price_lower = price[date2] < price[date1]  # Price making lower low
            macd_higher = macd[date2] > macd[date1]    # MACD making higher low
            
            if price_lower and macd_higher:
                bullish_div.append(date2)
    
    # Detect bearish divergence (price higher highs, MACD lower highs)
    peak_dates = price[price_peaks].index
    for i in range(1, len(peak_dates)):
        date1, date2 = peak_dates[i-1], peak_dates[i]
        
        # Check if MACD peaks exist at similar dates
        if date1 in macd.index and date2 in macd.index:
            price_higher = price[date2] > price[date1]  # Price making higher high
            macd_lower = macd[date2] < macd[date1]      # MACD making lower high
            
            if price_higher and macd_lower:
                bearish_div.append(date2)
    
    return {
        'bullish': bullish_div,
        'bearish': bearish_div,
        'price_peaks': price[price_peaks],
        'price_troughs': price[price_troughs],
        'macd_peaks': macd[macd_peaks],
        'macd_troughs': macd[macd_troughs]
    }


# Detect divergence
divergence = detect_divergence(maybank['Close'], macd_data['MACD'], order=5)

print("MACD Divergence Detection:")
print("="*70)
print(f"\n🟢 Bullish Divergence Points (Potential Bottoms): {len(divergence['bullish'])}")
if divergence['bullish']:
    for date in divergence['bullish']:
        print(f"   {date.strftime('%Y-%m-%d')} - Price: RM {maybank.loc[date, 'Close']:.2f}")

print(f"\n🔴 Bearish Divergence Points (Potential Tops): {len(divergence['bearish'])}")
if divergence['bearish']:
    for date in divergence['bearish']:
        print(f"   {date.strftime('%Y-%m-%d')} - Price: RM {maybank.loc[date, 'Close']:.2f}")

if not divergence['bullish'] and not divergence['bearish']:
    print("\n⚠️  No clear divergence detected in this time period.")
    print("   Try a different stock or longer time period.")

In [ ]:
# Visualize divergence
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Panel 1: Price with peaks and troughs
axes[0].plot(maybank.index, maybank['Close'], color='black', linewidth=1.5, label='Price')

# Mark price peaks and troughs
axes[0].scatter(divergence['price_peaks'].index, divergence['price_peaks'], 
                color='blue', marker='v', s=100, zorder=5, label='Price Peaks')
axes[0].scatter(divergence['price_troughs'].index, divergence['price_troughs'], 
                color='blue', marker='^', s=100, zorder=5, label='Price Troughs')

# Mark divergence points
for date in divergence['bullish']:
    axes[0].scatter(date, maybank.loc[date, 'Close'], 
                    color='green', marker='o', s=300, zorder=10, 
                    edgecolors='black', linewidths=2, label='Bullish Divergence')

for date in divergence['bearish']:
    axes[0].scatter(date, maybank.loc[date, 'Close'], 
                    color='red', marker='o', s=300, zorder=10,
                    edgecolors='black', linewidths=2, label='Bearish Divergence')

axes[0].set_ylabel('Price (RM)', fontsize=11)
axes[0].set_title('Price with Divergence Points', fontsize=12, fontweight='bold')
axes[0].legend(loc='best', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Panel 2: MACD with peaks and troughs
axes[1].plot(macd_data.index, macd_data['MACD'], color='blue', linewidth=1.5, label='MACD')
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)

# Mark MACD peaks and troughs
axes[1].scatter(divergence['macd_peaks'].index, divergence['macd_peaks'], 
                color='purple', marker='v', s=100, zorder=5, label='MACD Peaks')
axes[1].scatter(divergence['macd_troughs'].index, divergence['macd_troughs'], 
                color='purple', marker='^', s=100, zorder=5, label='MACD Troughs')

# Mark divergence points
for date in divergence['bullish']:
    if date in macd_data.index:
        axes[1].scatter(date, macd_data.loc[date, 'MACD'], 
                        color='green', marker='o', s=300, zorder=10,
                        edgecolors='black', linewidths=2)

for date in divergence['bearish']:
    if date in macd_data.index:
        axes[1].scatter(date, macd_data.loc[date, 'MACD'], 
                        color='red', marker='o', s=300, zorder=10,
                        edgecolors='black', linewidths=2)

axes[1].set_ylabel('MACD Value', fontsize=11)
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_title('MACD with Divergence Points', fontsize=12, fontweight='bold')
axes[1].legend(loc='best', fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Divergence Interpretation:")
print("="*70)
print("🟢 Green circles: Bullish divergence (price down, MACD up) → Potential BUY")
print("🔴 Red circles: Bearish divergence (price up, MACD down) → Potential SELL")
print("\n⚠️  Important: Divergence is an EARLY WARNING, not a guaranteed signal!")
print("   - Wait for confirmation (e.g., MACD crossover)")
print("   - Use with support/resistance levels")
print("   - Not all divergences lead to reversals")

## 6. Putting It All Together: Complete MACD Analysis

Let's create a comprehensive MACD trading system that combines:
1. **Crossover signals** (MACD vs Signal)
2. **Histogram momentum** (accelerating/decelerating)
3. **Zero-line crosses** (trend changes)
4. **Divergence detection** (reversal warnings)

### Trading Rules:

**Strong Buy Signal**:
- ✅ MACD crosses above Signal (bullish crossover)
- ✅ MACD > 0 (in uptrend)
- ✅ Histogram accelerating (momentum building)

**Strong Sell Signal**:
- ✅ MACD crosses below Signal (bearish crossover)
- ✅ MACD < 0 (in downtrend)
- ✅ Histogram accelerating downward (momentum building)

**Warning Signals**:
- ⚠️ Bullish divergence + MACD < 0 → Potential reversal up
- ⚠️ Bearish divergence + MACD > 0 → Potential reversal down

---

In [ ]:
# Complete MACD trading system
def macd_trading_signals(macd_data, crossovers, divergence):
    """
    Generate comprehensive MACD trading signals.
    
    Returns:
    --------
    pd.DataFrame : Signals with strength ratings
    """
    signals = pd.DataFrame(index=macd_data.index)
    signals['Signal'] = ''
    signals['Strength'] = 0
    signals['Reason'] = ''
    
    for date in macd_data.index:
        reasons = []
        strength = 0
        signal_type = ''
        
        # Get current values
        macd_val = macd_data.loc[date, 'MACD']
        signal_val = macd_data.loc[date, 'Signal']
        hist_val = macd_data.loc[date, 'Histogram']
        
        # Check for bullish crossover
        if date in crossovers[crossovers['Signal_Type'] == 'Bullish'].index:
            signal_type = 'BUY'
            strength += 2
            reasons.append('Bullish crossover')
            
            # Bonus if MACD > 0 (in uptrend)
            if macd_val > 0:
                strength += 1
                reasons.append('Already in uptrend')
        
        # Check for bearish crossover
        elif date in crossovers[crossovers['Signal_Type'] == 'Bearish'].index:
            signal_type = 'SELL'
            strength += 2
            reasons.append('Bearish crossover')
            
            # Bonus if MACD < 0 (in downtrend)
            if macd_val < 0:
                strength += 1
                reasons.append('Already in downtrend')
        
        # Check for divergence
        if date in divergence['bullish']:
            if signal_type == '':  # No crossover yet
                signal_type = 'WATCH (Bullish Divergence)'
            strength += 1
            reasons.append('Bullish divergence detected')
        
        if date in divergence['bearish']:
            if signal_type == '':  # No crossover yet
                signal_type = 'WATCH (Bearish Divergence)'
            strength += 1
            reasons.append('Bearish divergence detected')
        
        # Record signal
        if signal_type != '':
            signals.loc[date, 'Signal'] = signal_type
            signals.loc[date, 'Strength'] = strength
            signals.loc[date, 'Reason'] = '; '.join(reasons)
    
    # Filter to only rows with signals
    signals = signals[signals['Signal'] != '']
    
    return signals


# Generate signals
trading_signals = macd_trading_signals(macd_data, macd_with_signals, divergence)

print("Complete MACD Trading Signals:")
print("="*80)
print(trading_signals.to_string())

print("\n📊 Signal Strength Interpretation:")
print("="*80)
print("Strength 3+: STRONG signal (crossover + trend alignment)")
print("Strength 2: MODERATE signal (crossover only)")
print("Strength 1: WEAK signal (divergence warning)")

## 7. Exercises

Test your understanding with these exercises!

---

### Exercise 1: Calculate MACD for Top Glove

**Task**: Download Top Glove (5225.KL) data for 2023 and calculate the MACD components. Then answer:

1. How many bullish crossovers occurred?
2. How many bearish crossovers occurred?
3. Was the stock mostly in an uptrend or downtrend? (Check MACD sign)
4. Create a visualization showing all three MACD components

**Hint**: Use the `calculate_macd()` and `detect_macd_crossovers()` functions we created!

---

In [ ]:
# Your code here


### Exercise 2: Custom MACD Parameters

**Task**: The standard MACD uses (12, 26, 9) periods. Try creating a **faster MACD** with (5, 13, 5) periods and a **slower MACD** with (26, 52, 18) periods.

Using Maybank data:
1. Calculate all three MACD versions
2. Plot them on the same chart (normalized for comparison)
3. Which version generates more signals?
4. Which version has fewer false signals?

**Discussion**: What are the tradeoffs between fast and slow MACD settings?

---

In [ ]:
# Your code here


### Exercise 3: Histogram Momentum Strategy

**Task**: Create a trading strategy based ONLY on histogram momentum:

**Rules**:
- Buy when: Histogram crosses from negative to positive AND histogram is accelerating
- Sell when: Histogram crosses from positive to negative AND histogram is accelerating downward

Using CIMB (1023.KL) data for 2023:
1. Implement this strategy
2. Count buy and sell signals
3. Visualize the signals on a price chart
4. Compare with standard MACD crossover signals - which generates more signals?

---

In [ ]:
# Your code here


### Exercise 4: Multi-Stock MACD Dashboard

**Task**: Create a MACD dashboard for multiple Malaysian stocks.

For these stocks: Maybank (1155.KL), Top Glove (5225.KL), CIMB (1023.KL), Axiata (6888.KL)

Create a summary table showing (for the most recent date):
1. Current MACD value
2. Current Signal value
3. Current Histogram value
4. Trend (Bullish if MACD > 0, Bearish if MACD < 0)
5. Momentum state (from histogram analysis)
6. Latest signal (if any in the last 10 days)

**Bonus**: Sort the table by signal strength to find the best trading opportunities!

---

In [ ]:
# Your code here


---

## 📚 Summary

Congratulations! You now understand the **complete mathematics** behind MACD. Let's recap:

### Key Concepts:

1. **MACD Line** = EMA(12) - EMA(26)
   - Captures both trend (sign) and momentum (magnitude)
   - Positive → Uptrend, Negative → Downtrend

2. **Signal Line** = EMA(9) of MACD
   - Acts as a trigger for trading signals
   - Crossovers generate buy/sell signals

3. **Histogram** = MACD - Signal
   - Visual representation of convergence/divergence
   - Shows momentum acceleration/deceleration

4. **Divergence** = Price and MACD moving in opposite directions
   - Early warning of potential reversals
   - Bullish divergence → Potential bottom
   - Bearish divergence → Potential top

### Trading Signals:

| Signal Type | Condition | Strength |
|-------------|-----------|----------|
| **Strong Buy** | MACD crosses above Signal, MACD > 0, Histogram accelerating | ⭐⭐⭐ |
| **Moderate Buy** | MACD crosses above Signal | ⭐⭐ |
| **Weak Buy** | Bullish divergence (warning) | ⭐ |
| **Strong Sell** | MACD crosses below Signal, MACD < 0, Histogram accelerating down | ⭐⭐⭐ |
| **Moderate Sell** | MACD crosses below Signal | ⭐⭐ |
| **Weak Sell** | Bearish divergence (warning) | ⭐ |

### What You've Learned:

✅ Calculate MACD from scratch using EMA differences  
✅ Understand why MACD captures BOTH trend and momentum  
✅ Interpret Signal Line crossovers as trading signals  
✅ Use Histogram to gauge momentum acceleration  
✅ Detect divergence as early reversal warnings  
✅ Build a complete MACD trading system  

### ⚠️ MACD Limitations:

- **Lagging indicator**: Based on EMAs (which lag price)
- **Whipsaws**: Can give false signals in sideways markets
- **No overbought/oversold levels**: Unlike RSI or Stochastic
- **Best in trending markets**: Weak in consolidation periods

### 🎯 Best Practices:

1. **Combine with trend confirmation** (e.g., moving averages)
2. **Use divergence as early warning**, wait for crossover confirmation
3. **Adjust parameters** for different timeframes (faster for day trading, slower for swing trading)
4. **Consider market context**: Is the market trending or consolidating?
5. **Always use stop-losses**: No indicator is 100% accurate

---

## 🔜 What's Next?

In **Module 08: Correlation and Relationships**, you'll learn:
- How to measure correlation between stocks mathematically
- Analyzing sector relationships in KLSE (banking, gloves, tech)
- Portfolio diversification using correlation coefficients
- Creating correlation matrices and heatmaps

**Ready?** Move on to Module 08 when you can:
- ✅ Calculate MACD, Signal, and Histogram from scratch
- ✅ Explain why MACD captures both trend and momentum
- ✅ Identify crossover signals and divergence
- ✅ Complete all exercises without looking at solutions

---

### 📖 Additional Resources:

- [MACD on Investopedia](https://www.investopedia.com/terms/m/macd.asp)
- [Understanding MACD Divergence](https://www.investopedia.com/articles/forex/05/macddiverge.asp)
- [MACD Histogram Secrets](https://www.investopedia.com/articles/technical/091001.asp)

---

**Great work!** You now have a deep mathematical understanding of one of the most popular technical indicators. 🎉
